## Top Models
- Train many quick and dirty models
- Measure and compare performance using K-Fold cross-validation
- Shortlist the top four most promising models

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from src.topmodels import top_models

In [ ]:
# Try different models using k-fold cross validation and looking at the error on the test set too
top_models()

### Conclusions
The most promising models are 